<a href="https://colab.research.google.com/github/anuppohekar/Deep-Learning-Projects/blob/main/Day3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Step 1 - load data

In [ ]:
from keras.datasets import mnist

#Load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#Step 2: Reshape and standardize

In [ ]:
#60000 - 28*28*1 (Width * height * channel)
#channels - 1 for black and white, 3 for color (RGB)
#Each image a photo of a digit between 0-9
#Classes-10 ; multiclass
#train - 60000
#test - 10000

X_train = X_train.reshape((60000,28*28*1))
X_test = X_test.reshape((10000,28*28*1))
X_train = X_train/255.0
X_test = X_test/255.0



#Step 3: one hot encode y

In [ ]:
from keras.utils import to_categorical
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


#Building network

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

mnistnn = Sequential()
mnistnn.add(Dense(units=512, activation='relu', input_dim=784))
mnistnn.add(Dense(units=10, activation='softmax'))


In [ ]:
mnistnn.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
#callbacks
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

rd = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto')
es = EarlyStopping(monitor='val_loss', patience=8, verbose=1, mode='auto')
filepath = 'weight.{epoch:02d}.{val_loss:.2f}.h5'
mc = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')

In [ ]:
history = mnistnn.fit(x=X_train, y=y_train, epochs=10, validation_split=0.2, callbacks=[rd, es, mc])

#Step 5: Performance curves

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Performance curve')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train','Validation'])
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Performance curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train','Validation'])
plt.show()

#Step6: Evaluate

In [ ]:
mnistnn.evaluate(X_test, y_test)

#Train acc = 0.9978
#Val acc   = 0.9755
#Test acc =  0.9781

In [ ]:
from keras.models import load_model

newmodel = load_model('/content/weight.03.0.08.h5')

In [ ]:
newmodel.evaluate(X_test, y_test)

#Train acc - 1
#Val acc - 0.9812
#Test acc - 0.9811

#Project

In [ ]:
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template

#Website

In [ ]:
htmlpage = '''
<html>
<head>
<title>My first Project</title>
</head>

<body>

<form action="" method=post enctype=multipart/form-data>
<input type=file name=photo>
<input type=submit value=upload>
</form>

{{label}}
</body>

</html>
'''

!mkdir '/content/templates'
!mkdir '/content/uploads'


#Integrate model with index.html

In [ ]:
import os
from keras.preprocessing import image
from keras.preprocessing.image import load_img, img_to_array
import numpy as np


In [ ]:
#Give name
app = Flask(__name__)
run_with_ngrok(app)

#Create homepage
@app.route('/')
def homepage():
  return render_template('index.html')

#Predictions
@app.route('/', methods=['POST'])
def predictions():
  #Accept file from webpage
  img = request.files['photo']
  #save file 
  filepath=os.path.join('uploads',img.filename)
  img.save(filepath)
  #load file
  img = image.load_img(filepath, target_size=(28,28), color_mode='grayscale')
  #convert to numpy
  img = img_to_array(img)
  #reshape and standardize
  img = img.reshape((1,784))
  img = img/255.0
  #predict
  preds = newmodel.predict_classes(img)
  return render_template('index.html', label=preds)


#Execute app
if __name__=='__main__':
  app.run()

#flow_from_directory